In [3]:
import requests
def get_agol_token(username: str, password: str) -> str:
    """
    Generates an authentication token for ArcGIS Online using a username and password.

    Args:
        username (str): ArcGIS Online username
        password (str): ArcGIS Online password

    Returns:
        str: A valid authentication token used to make authorized API requests.

    Raises:
        ValueError: If authentication fails or token missing from response.
        ConnectionError: If requests cannot reach the AGOL endpoint.
    """

    url = "https://www.arcgis.com/sharing/rest/generateToken"

    data = {
        "username": username,
        "password": password,
        "referer": "https://www.arcgis.com",
        "f": "json"
    }

    try:
        response = requests.post(url, data=data)

        if response.status_code != 200:
            raise Exception(f"Request failed with status code {response.status_code}: {response.text}")

        token_data = response.json()

        if "token" in token_data:
            return token_data["token"]
        elif "error" in token_data:
            raise ValueError(f"Authentication failed: {token_data['error']['message']}")
        else:
            raise ValueError("Unexpected response format: Token not found.")

    except requests.exceptions.RequestException as e:
        raise ConnectionError(f"Failed to connect to ArcGIS Online: {e}")


In [ ]:
import requests
import json
token = get_agol_token("AKDOT_APEX", "@KD0T_@p3x")

url = 'https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/Pavement_Condition_Data_Tenth_Mile_2024/FeatureServer/0/query'
layer = 0

route_id = '1160000X000'
route = 'Glenn Highway'
from_mp = 166.8
to_mp = 166.9


where = ( f"ROUTE_NAME = '{route}' " f"AND FROM_MPT >= {from_mp} " f"AND TO_MPT <= {to_mp}" )

params = { "where": where, 
          "outFields": '*', 
          "returnGeometry": 'true', 
          "outSR": 4326, 
          "f": "json", 
          "token": token }

response = requests.get(url, params=params)

if response.status_code != 200:
    raise Exception(
        f"Request failed with status code {response.status_code}: {response.text}"
    )

data = response.json()['features']

if "error" in data:
    raise Exception(
        f"API Error: {data['error']['message']} - {data['error'].get('details', [])}"
    )


# your features list is called "features"
features = data   # replace with your list

# Step 1: extract all paths from all features
all_paths = []
for f in features:
    for path in f["geometry"]["paths"]:
        all_paths.extend(path)

# Step 2: remove consecutive duplicates while preserving order
combined_path = []
seen = set()

for pt in all_paths:
    tup = tuple(pt)
    if tup not in seen:
        combined_path.append(pt)
        seen.add(tup)

# combined_path now contains a single merged path with no repeats
print("Total points:", len(combined_path))
print(combined_path)


Total points: 18
[[-145.869744413782, 62.1063239572288], [-145.871294808076, 62.1063097361806], [-145.87179945971, 62.1063052569429], [-145.872834197409, 62.1062949828564], [-145.869782839842, 62.106286439478], [-145.869021299702, 62.1062922333089], [-145.867974675363, 62.1063015728249], [-145.867547448944, 62.106305670862], [-145.866688484423, 62.1063143624491], [-145.866655590878, 62.1063542905651], [-145.867982421353, 62.1063409490747], [-145.868703928229, 62.1063344276752], [-145.863571575829, 62.1063840120493], [-145.864118958397, 62.1063790540772], [-145.864578192308, 62.1063752243233], [-145.864762156197, 62.1063735335954], [-145.86521284878, 62.1063690700302], [-145.865998455733, 62.1063608393898]]
